In [36]:
# Importing required libraries for the analysis of imagery data
# Operating system dependent functionality
import os
# Data processing
import numpy as np 
import pandas as pd 
# Plotting
import plotly.express as px
import plotly.subplots as sp
# Image processing
import cv2 as cv
from PIL import Image
# Keras
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from keras.models import load_model
from keras.utils import to_categorical
# Quality checking
from sklearn.metrics import accuracy_score

In [37]:
# based on https://www.kaggle.com/code/lalithmovva/99-accuracy-on-german-traffic-sign-recognition
# Reading the input images and putting them into a numpy array
data = []
labels = []
height = 50
width = 50
channels = 3
classes = 43
n_inputs = height * width * channels

my_information = {
                0: '20km', 1: '30km', 
                2: '50km', 3: '60km', 
                4: '70km', 5: '80km',
                6: '80km_crossed', 7: '100km',
                8: '120km', 9: 'no_overtaking', 
                10: 'no_overtaking_for_tracks', 11: 'crossroads',
                12: 'priority_roads', 13: 'give_way', 
                14: 'stop', 15: 'road_closed', 
                16: 'no_tractor', 17: 'no_entry',
                18: 'attention', 19: 'sharp_curve_left', 
                20: 'sharp_curve_right', 21: 'zigzag_left',
                22: 'rough_road', 23: 'car_skid',
                24: 'road_narrowing_right', 25: 'road_works', 
                26: 'traffic_light', 27: 'crosswalk', 
                28: 'children_crossing', 29: 'bike_road',
                30: 'snow', 31: 'wild_animal', 
                32: 'derestriction', 33: 'mandatory_right_turn', 
                34: 'mandatory_left_turn', 35: 'mandatory_straight_ahead',
                36: 'mandatory_straight_ahead_or_turning_right', 37: 'mandatory_straight_ahead_or_turning_left', 
                38: 'mandatory_passing_right', 39: 'mandatory_passing_left', 
                40: 'mandatory_direction_of_the_roundabout', 41: 'derestriction_of_overtaking', 
                42: 'derestriction_of_overtaking_for_tracks'}

In [41]:
# based on https://www.kaggle.com/code/lalithmovva/99-accuracy-on-german-traffic-sign-recognition
for i in range(classes) :
    path = "my_path/Imagery_data_GTSRB/Train/{0}/".format(i)
    Class = os.listdir(path)
    for a in Class:
        image = cv.imread(path+a)
        imageRGB = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image_from_array = Image.fromarray(imageRGB)
        size_image = image_from_array.resize((height, width))
        data.append(np.array(size_image))
        labels.append(i)
Cells = np.array(data)
labels = np.array(labels)
# Randomize the order of the input images
s = np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells = Cells[s]
labels = labels[s]

In [42]:
# code snippet from https://www.kaggle.com/code/lalithmovva/99-accuracy-on-german-traffic-sign-recognition
# Spliting the images into train and validation sets
(X_train,X_val)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))] # 80% train (31368 images), 20% validation (7841 images)
X_train = X_train.astype('float32')/255 
X_val = X_val.astype('float32')/255
(y_train,y_val)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)

Training

In [38]:
# based on https://www.kaggle.com/code/lalithmovva/99-accuracy-on-german-traffic-sign-recognition
model = Sequential()
x = Conv2D(filters=64,kernel_size=(5,5),input_shape=X_train.shape[1:])
model.add(x)
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))
model.compile(
    loss='categorical_crossentropy', 
    optimizer='Adam', 
    metrics=['accuracy']
)
earlyStopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, mode='max')
mcp_save = ModelCheckpoint('best_model.mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')
epochs = 100
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val), callbacks=[earlyStopping, mcp_save])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        4864      
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 22, 22, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                   

Loading the exising model

In [39]:
model = load_model('my_path/cnn.mdl_wts.hdf5')

In [ ]:
# based on https://www.kaggle.com/code/lalithmovva/99-accuracy-on-german-traffic-sign-recognition
# Predicting with the test data
y_test = pd.read_csv("my_path/Imagery_data_GTSRB/Test.csv")
labels = y_test['Path'].to_numpy()
y_test = y_test['ClassId'].values
data=[]
for f in labels:
    image = cv.imread('my_path/Imagery_data_GTSRB/Test/'+f.replace('Test/', ''))
    imageRGB = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image_from_array = Image.fromarray(imageRGB)
    resized_image = image_from_array.resize((height, width))
    data.append(np.array(resized_image))
X_test=np.array(data)
X_test = X_test.astype('float32')/255 
predict_x = model.predict(X_test) 
classes_x = np.argmax(predict_x,axis=1)
print("The accuracy of the CNN classifier on the test set is: ", accuracy_score(classes_x, y_test))

395/395 [==============================] - 7s 19ms/step
The accuracy of the CNN classifier on the test set is:  0.9733174980205859


In [ ]:
fig1 = px.line(y=[model.history.history['accuracy'], model.history.history['val_accuracy']],
                 width=1000, height=1000).update_layout(
                 xaxis_title="epoch", yaxis_title="accuracy")

fig1.update_layout(
    font=dict(size=25),
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)
fig1.update_traces(line=dict(width=5.5))
newnames = {'wide_variable_0':'train', 'wide_variable_1': 'validation'}
fig1.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )

In [ ]:
fig2 = px.line(y=[model.history.history['loss'], model.history.history['val_loss']],
                 width=1000, height=1000).update_layout(
                 xaxis_title="epoch", yaxis_title="loss")

fig2.update_layout(
    font=dict(size=25),
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)
fig2.update_traces(line=dict(width=5.5))
newnames = {'wide_variable_0':'train', 'wide_variable_1': 'validation'}
fig2.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )

In [ ]:
# Create figures in Express
figure1 = fig1
figure2 = fig2

# For as many traces that exist per Express figure, get the traces from each plot and store them in an array.
# This is essentially breaking down the Express fig into it's traces
figure1_traces = []
figure2_traces = []
for trace in range(len(figure1["data"])):
    figure1_traces.append(figure1["data"][trace])
for trace in range(len(figure2["data"])):
    figure2_traces.append(figure2["data"][trace])

#Create a 1x2 subplot
this_figure = sp.make_subplots(rows=1, cols=2) 

# Get the Express fig broken down as traces and add the traces to the proper plot within in the subplot
for traces in figure1_traces:
    this_figure.append_trace(traces, row=1, col=1)
for traces in figure2_traces:
    this_figure.append_trace(traces, row=1, col=2)

this_figure.update_layout(
    font=dict(size=25),
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
)
this_figure.update_layout(
                 xaxis_title="epoch", yaxis_title="acccuracy")
this_figure.update_layout(
                 xaxis2_title="epoch", yaxis2_title="loss")
this_figure.update_layout(width=2000, height=1000)
this_figure.update_layout(showlegend=False)
this_figure.show(showlegend=False)